In [1]:
# !pip install flask-ngrok
# !pip install flask_ngrok
# !pip install pyngrok
# !ngrok authtoken 2EvTUNCqpWpOeoHXGnEqsj1f2Jd_52sEd1imYByeKph9bPSMH
# !pip install ftfy==6.1.1
# !pip install transformers==4.20.1
# !pip install tensorflow_addons==0.17.1

In [2]:
#from google.colab import drive
import numpy as np
import pickle
import tensorflow as tf
import random
import pandas as pd
import warnings,operator
from tqdm import tqdm
import time
import os, re, string
import time

warnings.filterwarnings("ignore")
from transformers import logging
from transformers import TFAutoModel, AutoTokenizer
from transformers import RobertaConfig, TFXLMRobertaModel, RobertaTokenizer,TFRobertaModel,TFBertModel
logging.set_verbosity_error()
from tensorflow.keras.utils import pad_sequences
#from google.colab import drive
#drive.mount('/content/gdrive')




In [3]:
import string,re
def clean_doc(text):
  text = text.lower()
  text = text.translate(text.maketrans('', '', string.punctuation))
  text = re.sub(r"[0-9]+", " num ", text)
  text = re.sub('\\s+',' ',text)
  text = text.strip()
  return text

In [4]:
listLabel = "RESTAURANT#GENERAL,SERVICE#GENERAL,FOOD#QUALITY,FOOD#STYLE_OPTIONS,DRINKS#STYLE_OPTIONS,DRINKS#PRICES,RESTAURANT#PRICES,RESTAURANT#MISCELLANEOUS,AMBIENCE#GENERAL,FOOD#PRICES,LOCATION#GENERAL,DRINKS#QUALITY"
categories = listLabel.split(',')

In [5]:
print("Load English model")
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name, do_lower_case=False)

Load English model


In [6]:
# Load RoBertTa model

start = time.time()
model = tf.keras.models.load_model( "./model/RoBertTa.h5",custom_objects={"TFRobertaModel": TFRobertaModel})
end = time.time()
print("Time for Loading model: " + str(end - start))

2022-09-26 15:16:28.062065: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
#!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
#!tar -xvf /content/ngrok-stable-linux-amd64.tgz
#!./ngrok authtoken 2EvTUNCqpWpOeoHXGnEqsj1f2Jd_52sEd1imYByeKph9bPSMH

In [ ]:
# !cp -r "/content/gdrive/MyDrive/ABSA_English/DemoWeb/" "DemoWeb"
# %cd "DemoWeb"
# !ls

In [ ]:

from flask import Flask, render_template,request,send_file,Response
import pickle
import numpy as np
import os, string, re
from flask_ngrok import run_with_ngrok
import os
app = Flask(__name__)
# run_with_ngrok(app)  


# @app.route("/")
# def home():
#     return render_template("home.html")

# @app.route("/sentiment/", methods=['POST','GET'])
def sentiment(text):
    query_clean = clean_doc(text)
    encoding_demo = tokenizer([query_clean], return_tensors='tf', padding="max_length", truncation=True, max_length=74)
    demo_ids = encoding_demo['input_ids']
    demo_attention_mask = encoding_demo['attention_mask']
    predicted = model.predict([np.expand_dims(demo_ids[0], axis=0),np.expand_dims(np.array(demo_attention_mask[0]), axis=0)])
    s = ''
    for i, predict in enumerate(predicted):
        index2, value = max(enumerate(predict[0]), key=operator.itemgetter(1))
        if index2 == 1:
            s+= str(categories[i]).split('#')[0] + ': positive  <br>  '
        elif index2 == 2:
            s+= str(categories[i]).split('#')[0] + ': neutral  <br>  '
        elif index2 == 3:
            s+= str(categories[i]).split('#')[0] + ': negative  <br> '

    label = s[:-2]
    print(s)
    print("Result : ", label)

In [ ]:
a = "Foods are okay, the place look modern and clean"
sentiment(a)

In [ ]:
b = "FOOD#QUALITY"
b.split('#')[0]